In [19]:

cost = df = pd.read_excel(r"C:\Users\Dell5559\Desktop\python\cost.xlsx",nrows=6)
print(cost)


  From/To    K1    K2    K3    D1    D2    D3
0      P1  1.61  2.12  3.58  1.04  1.39  1.34
1      P2  3.77  2.89  2.65  1.41  1.14  1.36
2      D1  0.61  0.61  0.58   NaN   NaN   NaN
3      D2  0.38  0.56  0.76   NaN   NaN   NaN
4      D3  0.41  0.58  0.35   NaN   NaN   NaN


In [7]:
from pulp import *
import pandas as pd
problem = pulp.LpProblem("LPProblem", pulp.LpMinimize)



production_plants= ['P1', 'P2']
distribution_centers= ['D1', 'D2', 'D3']
customers = ['K1', 'K2', 'K3']


#SET PARAMETERS
plants_capacity = dict(zip(production_plants,[45000,28500]))

distribution_capacity = dict(zip(distribution_centers, [75000, 75000, 75000]))

demand= dict(zip(customers, [14740, 27830,19990]))

fix_cost_DC = dict(zip(distribution_centers, [20000,20000,20000]))


        #Transport cost/unit from production plants to distribution centers
C_pd = dict(zip(production_plants,[dict(zip(distribution_centers, [1.04, 1.39, 1.34])), 
                                   dict(zip(distribution_centers, [1.41, 1.14, 1.36]))]))
        #Transport cost/unit from distribution_centers to customers
C_dk = dict(zip(distribution_centers,[dict(zip(customers, [0.61,0.61,0.58])), 
                                      dict(zip(customers, [0.38,0.56,0.76])),
                                      dict(zip(customers, [0.41,0.58,0.35]))]))
        #Transport cost/unit from production plants to customers
C_pk = dict(zip(production_plants,[dict(zip(customers, [1.61,2.12,3.58])), 
                                   dict(zip(customers, [3.77,2.89,2.65]))]))


#SET DECISION VARIABLES
X_pd = LpVariable.dicts('Transport from P to D', [(p,d) for p in production_plants for d in distribution_centers], lowBound=0, cat=
'Integer')
X_dk = LpVariable.dicts('Transport from D to K', [(d,k) for k in customers for d in distribution_centers], lowBound=0, cat=
'Integer')
X_pk = LpVariable.dicts('Transport from P to K', [(p,k) for p in production_plants for k in customers], lowBound=0, cat=
'Integer')
use_vars = LpVariable.dicts('Use or not', distribution_centers, lowBound=0, upBound=1, cat='Integer')
#Set MINIMIZATION OBJECTIVE FUNCTION
prob  =  LpProblem("Min_Cost", LpMinimize)
tmp1 = sum(fix_cost_DC[d] * use_vars[d] for d in distribution_centers) 
tmp2 = sum(sum(C_pd[p][d] * X_pd[(p,d)] for p in production_plants) for d in distribution_centers)
tmp3 = sum(sum(C_dk[d][k] * X_dk[(d,k)] for d in distribution_centers) for k in customers)
tmp4 = sum(sum(C_pk[p][k] * X_pk[(p,k)] for p in production_plants) for k in customers)
prob += tmp1+tmp2+tmp3+tmp4

#SET CONSTRAINTS
for k in customers:
    prob += lpSum(X_dk[(d,k)] for d in distribution_centers)+ lpSum(X_pk[(p,k)] for p in production_plants) == demand[k] #DEMAND

for d in distribution_centers:
    prob += lpSum(X_pd[(p,d)] for p in production_plants) == lpSum(X_dk[(d,k)] for k in customers)
    prob += lpSum(X_pd[(p,d)] for p in production_plants) <= distribution_capacity[d]
    
for p in production_plants:
    prob += lpSum(X_pd[(p,d)] for d in distribution_centers) + lpSum(X_pk[(p,k)] for k in customers) <= plants_capacity[p]
    
for p in production_plants:
    for d in distribution_centers :
        prob += X_pd[(p,d)] <= use_vars[d]*10000000000000



prob.solve()
print("Status: ", LpStatus[prob.status])
TOL= 0.00001
for d in distribution_centers:
    if use_vars[d].varValue > TOL:
        print("Establish Distribution Center at: " + d)
    
    


print("Objective: ",value(prob.objective))
x = (1-value(prob.objective)/175318.6)*100

print("Save in Percent: " ,round(x,1), "\n\n")
for v in prob.variables():
    print (v.name , "=", v.varValue)


Status:  Optimal
Establish Distribution Center at: D1
Objective:  128531.9
Save in Percent:  26.7 


Transport_from_D_to_K_('D1',_'K1') = 0.0
Transport_from_D_to_K_('D1',_'K2') = 27830.0
Transport_from_D_to_K_('D1',_'K3') = 19990.0
Transport_from_D_to_K_('D2',_'K1') = 0.0
Transport_from_D_to_K_('D2',_'K2') = 0.0
Transport_from_D_to_K_('D2',_'K3') = 0.0
Transport_from_D_to_K_('D3',_'K1') = 0.0
Transport_from_D_to_K_('D3',_'K2') = 0.0
Transport_from_D_to_K_('D3',_'K3') = 0.0
Transport_from_P_to_D_('P1',_'D1') = 30260.0
Transport_from_P_to_D_('P1',_'D2') = 0.0
Transport_from_P_to_D_('P1',_'D3') = 0.0
Transport_from_P_to_D_('P2',_'D1') = 17560.0
Transport_from_P_to_D_('P2',_'D2') = 0.0
Transport_from_P_to_D_('P2',_'D3') = 0.0
Transport_from_P_to_K_('P1',_'K1') = 14740.0
Transport_from_P_to_K_('P1',_'K2') = 0.0
Transport_from_P_to_K_('P1',_'K3') = 0.0
Transport_from_P_to_K_('P2',_'K1') = 0.0
Transport_from_P_to_K_('P2',_'K2') = 0.0
Transport_from_P_to_K_('P2',_'K3') = 0.0
Use_or_not_D1 = 1.